In [35]:
import pandas as pd
from bs4 import BeautifulSoup

import sys
import os
import xml.etree.ElementTree as ET
from pathlib import Path

categoriesFilename = '/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml'

root_category_id = 'cat00000'

tree = ET.parse(categoriesFilename)
root = tree.getroot()

# Parse the category XML file to map each category id to its parent category id in a dataframe.
categories = []
parents = []
for child in root:
    id = child.find('id').text
    cat_path = child.find('path')
    cat_path_ids = [cat.find('id').text for cat in cat_path]
    leaf_id = cat_path_ids[-1]
    if leaf_id != root_category_id:
        categories.append(leaf_id)
        parents.append(cat_path_ids[-2])
parents_df = pd.DataFrame(list(zip(categories, parents)), columns =['category', 'parent'])

parents_df.head()

,category,parent
0,abcat0010000,cat00000
1,abcat0011000,abcat0010000
2,abcat0011001,abcat0011000
3,abcat0011002,abcat0011000
4,abcat0011003,abcat0011000


In [33]:
!grep touchpad /workspace/datasets/train.csv\
    | cut -d',' -f3 | \
    python leavesToPaths.py --max_depth 4 | sort | uniq -c | sort -nr | head

  16616 Best Buy > Computers & Tablets > Tablets & iPad > Tablets
   1136 Best Buy > Computers & Tablets > Tablets & iPad > Tablet Accessories
    169 Best Buy > Computers & Tablets > Tablets & iPad > iPad
    144 Best Buy > Computers & Tablets > Laptop & Netbook Computers > PC Laptops
     74 Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories
     66 Best Buy > Computers & Tablets > E-Readers > E-Reader Devices
     33 Best Buy > Appliances > Small Appliances > Blenders & Juicers
     32 Best Buy > TV & Home Theater > TVs > All Flat-Panel TVs
     31 Best Buy > Computers & Tablets > Mice & Keyboards > USB Tablets & Pens
     28 Best Buy > Computers & Tablets > Mice & Keyboards > Keyboards


In [3]:
queries = pd.read_csv('/workspace/datasets/train.csv').sample(10_000)
queries['path'] = queries['category'].map(catDict)
queries = queries.set_index('category').drop(['sku', 'user', 'query_time'], axis=1)
queries = queries[~queries.path.isna()].copy()
queries.head()

,query,click_time,path
category,,,
abcat0202003,hdmi receiver,2011-10-23 10:24:16.198,Best Buy > TV & Home Theater > Home Theater Sy...
pcmcat183800050007,Power cord,2011-09-13 10:43:30.66,Best Buy > Best Buy For Business > Small Busin...
abcat0301014,garmin,2011-09-29 18:38:10.518,"Best Buy > Car, Marine & GPS > GPS Navigation ..."
pcmcat209000050007,Apple iPad,2011-08-29 12:26:37.548,Best Buy > Computers & Tablets > Tablets & iPa...
abcat0202002,Stereo system,2011-10-12 19:05:27.919,Best Buy > Audio & MP3 > Home Audio > Stereo S...


In [7]:
queries['path_length'] = queries['path'].map(lambda x: len(x.split('>')))
sizes = queries.groupby('category').size()
queries['leaf_counts'] = queries.index.map(sizes)
queries.sample(10)

,query,click_time,path,path_length,leaf_counts
category,,,,,
pcmcat247400050001,Macbook pro,2011-09-09 10:26:49.154,Best Buy > Computers & Tablets > Laptop & Netb...,4,150
pcmcat144700050004,bose,2011-10-18 21:21:09.852,Best Buy > Audio & MP3 > Headphones > All Head...,4,221
abcat0503002,OnlineMidnightSale_Computers_Tablets,2011-10-27 01:13:36.391,Best Buy > Computers & Tablets > Networking & ...,5,77
abcat0201011,32 gb itouch white,2011-10-14 06:00:14.704,Best Buy > Appliances > Fitness > Biking > All...,5,132
pcmcat167300050040,Onkyo,2011-10-25 13:47:01.693,Best Buy > TV & Home Theater > Home Theater Sy...,4,76
abcat0911006,hoover tempo widepath,2011-10-26 14:30:56.522,Best Buy > Appliances > Vacuum Cleaners > Upri...,4,21
abcat0912022,toaster oven,2011-10-19 23:52:08.173,Best Buy > Appliances > Small Appliances > Toa...,4,8
pcmcat218000050003,Stylus iPad,2011-09-03 09:19:41.408,Best Buy > Computers & Tablets > Tablets & iPa...,5,13
abcat0201009,ipod classic,2011-10-16 00:33:11.535,Best Buy > Audio & MP3 > iPod & MP3 Players > ...,4,7


In [9]:
!head /workspace/datasets/train.csv\
    | cut -d',' -f3 | \
    python leavesToPaths.py --max_depth 6

Namespace(input='/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml', max_depth=6)
Best Buy > TV & Home Theater > TVs > All Flat-Panel TVs
Best Buy > TV & Home Theater > TVs > All Flat-Panel TVs
Best Buy > Computers & Tablets > E-Readers > E-Reader Devices
Best Buy > TV & Home Theater > TVs > All Flat-Panel TVs
Best Buy > TV & Home Theater > TVs > TV/DVD Combos
Best Buy > Magnolia Home Theater > Televisions
Best Buy > Computers & Tablets > Laptop & Netbook Computers > MacBooks
Best Buy > Mobile Phones > Mobile Phone Accessories > Headsets, Speakers & Transmitters
Best Buy > TV & Home Theater > TVs > Tube TVs > TV Accessories > TV Antennas


In [20]:
from bs4 import BeautifulSoup

In [26]:
with open(categoriesFilename, 'r') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')
    categories = soup.find_all('category')
    print(len(categories))

31117


In [30]:
categories[3]

<category>
<id>pcmcat140000050035</id>
<name>Capturing Photos &amp; Videos</name>
</category>

In [25]:
print(str(soup)[:1000])

<categories>
<category>
<id>abcat0010000</id>
<name>Gift Center</name>
<path>
<category>
<id>cat00000</id>
<name>Best Buy</name>
</category>
<category>
<id>abcat0010000</id>
<name>Gift Center</name>
</category>
</path>
<subcategories>
<category>
<id>pcmcat140000050035</id>
<name>Capturing Photos &amp; Videos</name>
</category>
<category>
<id>pcmcat140000050036</id>
<name>Listening to Digital Music</name>
</category>
<category>
<id>pcmcat140000050037</id>
<name>Computing Made Easy</name>
</category>
<category>
<id>pcmcat140000050039</id>
<name>Simple GPS Navigation</name>
</category>
<category>
<id>pcmcat140000050040</id>
<name>Playing Video Games</name>
</category>
<category>
<id>pcmcat140000050041</id>
<name>Watching HDTV</name>
</category>
<category>
<id>pcmcat140000050042</id>
<name>Enjoying Favorite Movies</name>
</category>
<category>
<id>abcat0012000</id>
<name>Him</name>
</category>
<category>
<id>abcat0011000</id>
<name>Her</name>
</category>
<category>
<id>abcat0013000</id>
<n